In [1]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

/home/kownse/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = './'
TRAIN = '../input/train_v2/'
TEST = '../input/test_v2/'
SEGMENTATION = '../input/train_ship_segmentations_v2.csv.zip'
exclude_list = ['6384c3e78.jpg','13703f040.jpg', '14715c06d.jpg',  '33e0ff2d5.jpg',
                '4d4e09f2a.jpg', '877691df8.jpg', '8b909bb20.jpg', 'a8d99130e.jpg', 
                'ad55c3143.jpg', 'c8260c541.jpg', 'd6c7f17c7.jpg', 'dc3e7c901.jpg',
                'e44dffe88.jpg', 'ef87bad36.jpg', 'f083256d8.jpg'] #corrupted image

In [3]:
nw = 4   #number of workers for data loader
arch = resnet34 #specify target architecture

In [4]:
test_names = [f for f in os.listdir(TEST)]
train_names = [f for f in os.listdir(TRAIN)]
for el in exclude_list:
    if(el in train_names): train_names.remove(el)
    if(el in test_names): test_names.remove(el)

tr_n, val_n = train_test_split(train_names, test_size=0.05, random_state=42)

folds = pd.read_csv('../input/folds.csv')

In [5]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz))
    
    def get_y(self, i):
        if(self.path == TEST): return 0
        masks = self.segmentation_df.loc[self.fnames[i]]['EncodedPixels']
        if(type(masks) == float): return 0 #NAN - no ship 
        else: return 1
    
    def get_c(self): return 2 #number of classes

In [6]:
def get_data(sz,bs,tr_n, val_n):
    #data augmentation
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
#     md.is_multi = False
    return md

def get_data_val(sz,bs,tr_n, val_n):
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(val_n,TRAIN))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
    return md

In [7]:
sz = 384 #image size
bs = 32  #batch size

for i in tqdm_notebook([0,1,2,3,4]):

    tr_n = folds.loc[folds.fold != i, 'ImageId'].values.tolist()
    val_n = folds.loc[folds.fold == i, 'ImageId'].values.tolist()
    
    for el in exclude_list:
        if(el in tr_n): tr_n.remove(el)
        if(el in val_n): val_n.remove(el)

    md = get_data(sz,bs,tr_n, val_n)
    learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.load('Resnet34_lable_384_fold{}_1'.format(i))
    
    log_preds,y = learn.predict_with_targs(is_test=True)
    probs = np.exp(log_preds)[:,1]
    
    df = pd.DataFrame({'id':test_names, 'p_ship':probs})
    df.to_csv('../result/ship_detection_fold{}.csv'.format(i), header=True, index=False)
    
    md = get_data_val(sz,bs,tr_n, val_n)
    learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.load('Resnet34_lable_384_fold{}_1'.format(i))
    
    log_preds,y = learn.predict_with_targs(is_test=True)
    probs = np.exp(log_preds)[:,1]
    
    df = pd.DataFrame({'id':val_n, 'p_ship':probs})
    df.to_csv('../result/oof_detection_fold{}.csv'.format(i), header=True, index=False)

/home/kownse/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
